Experiments with global optimisation

In [ ]:
import math
import time
import bisect

import pandas as pd
import numpy as np
import plotly as plt
import ray

pd.options.plotting.backend = "plotly"

import sys, os, os.path

sys.path.append(os.path.expanduser("../src"))

from generate_common import custom_ray_init, cache_load
from spinorama.filter_iir import Biquad
from spinorama.filter_peq import peq_print, peq_format_apo, peq_spl, Peq
import scipy.optimize as opt

In [ ]:
custom_ray_init({"--log-level": "INFO"})

In [ ]:
# speaker_name = "Genelec 8341A"
# speaker_origin = "ASR"
# speaker_version = "asr-vertical"
# speaker_name = "BIC America FH6-LCR Center"
# speaker_origin = "ASR"
# speaker_version = "asr-vertical"
# speaker_name = "Arendal Sound 1961 Center"
# speaker_name = "KEF R8 Meta"
# speaker_name = "JBL AC25"
speaker_origin = "ASR"
speaker_version = "asr"
speaker_name = "JBL Control X Wireless"
speaker_origin = "Misc"
speaker_version = "misc-matthews"
df_all_speaker = cache_load({"speaker_name": speaker_name, "origin": speaker_origin}, False)
ray.shutdown()

In [ ]:
df_speaker = df_all_speaker[speaker_name][speaker_origin][speaker_version]
spl_h = df_speaker["SPL Horizontal_unmelted"]
spl_v = df_speaker["SPL Vertical_unmelted"]

freq = spl_h.Freq
spl_h = spl_h.drop("Freq", axis=1)
spl_v = spl_v.drop("Freq", axis=1)
spl = np.concatenate((spl_h.T.to_numpy(), spl_v.T.to_numpy()), axis=0)

In [ ]:
%load_ext Cython

In [ ]:
import bisect
from spinorama.constant_paths import MIDRANGE_MAX_FREQ
from spinorama.ltype import Vector
from spinorama.auto_misc import get3db
from spinorama.auto_loss import score_loss

In [ ]:
_, freq_min = get3db(df_speaker, 3.0)

FREQ_NB_POINTS = 200

freq_max = 10000
freq_first = max(freq_min, 20)
freq_last = min(freq_max, 20000)
freq_low = bisect.bisect(freq, freq_first)
freq_high = bisect.bisect(freq, freq_last)
freq_midrange = bisect.bisect(freq, MIDRANGE_MAX_FREQ / 2)

lw = df_speaker["CEA2034_unmelted"]["Listening Window"].to_numpy()
target = lw[freq_low:freq_high] - np.linspace(0, 0.5, len(lw[freq_low:freq_high]))

log_freq = np.logspace(np.log10(20), np.log10(freq_max), FREQ_NB_POINTS + 1)
min_db = 1
max_db = 3
min_q = 1
max_q = 4
max_peq = 7
max_iter = 5000

In [ ]:
def x2peq(x: list[float | int]) -> Peq:
    l = len(x) // 4
    peq = []
    for i in range(l):
        ifreq = int(x[i * 4 + 1])
        peq_freq = log_freq[ifreq]
        peq_freq = max(freq_min, peq_freq)
        peq_freq = min(freq_max, peq_freq)
        peq.append((1.0, Biquad(int(x[i * 4]), int(peq_freq), 48000, x[i * 4 + 2], x[i * 4 + 3])))
    return peq


def x2spl(x: list[float | int]) -> Vector:
    return peq_spl(freq, x2peq(x))


def opt_peq_score(x) -> float:
    peq = x2peq(x)
    peq_freq = np.array(x2spl(x))[freq_low:freq_high]
    score = score_loss(df_speaker, peq)
    flat = np.add(target, peq_freq)
    # flatness_l2 = np.linalg.norm(flat, ord=2)
    # flatness_l1 = np.linalg.norm(flat, ord=1)
    flatness_bass_mid = np.linalg.norm(flat[0 : freq_midrange - freq_low], ord=2)
    flatness_mid_high = np.linalg.norm(flat[freq_midrange - freq_low :], ord=2)
    # this is black magic, why 10, 20, 40?
    # if you increase 20 you give more flexibility to the score (and less flat LW/ON)
    # without the constraint optimising the score get crazy results
    return score + float(flatness_bass_mid) / 5 + float(flatness_mid_high) / 50


def opt_peq_flat(x) -> float:
    peq_freq = np.array(x2spl(x))[freq_low:freq_high]
    flat = np.add(target, peq_freq)
    flatness_l2 = np.linalg.norm(flat, ord=2)
    flatness_l1 = np.linalg.norm(flat, ord=1)
    return float(flatness_l1 + flatness_l2)


def opt_bounds_all(n: int) -> list[list[int | float]]:
    bounds0 = [
        [0, 6],
        [0, FREQ_NB_POINTS],  # algo does not support log scaling so I do it manually
        [min_q, 1.3],  # need to be computed from max_db
        [-max_db, max_db],
    ]
    bounds1 = [
        [3, 3],
        [0, FREQ_NB_POINTS],
        [min_q, max_q],
        [-max_db, max_db],
    ]
    return bounds0 + bounds1 * (n - 2) + bounds0


def opt_bounds_pk(n: int) -> list[list[int | float]]:
    bounds0 = [
        [3, 3],
        [0, FREQ_NB_POINTS],
        [min_q, max_q],
        [-max_db, max_db],
    ]
    return bounds0 * n


def opt_integrality(n: int) -> list[bool]:
    return [True, True, False, False] * n


def opt_constraints(n: int):
    # Create some space between the various PEQ; if not the optimiser will add multiple PEQ
    # at more or less the same frequency and that will generate too much of a cut on the max
    # SPL. we have 200 points from 20Hz-20kHz, 5 give us 1/4 octave
    m = n
    mat = np.asarray([[0] * (n * 4)] * m)
    vec = np.asarray([0] * m)
    for i in range(m):
        if i == 0:
            # first freq can be as low as possible
            # second needs to be > freq_min
            mat[0][5] = -1
            vec[0] = -freq_min
            continue
        j = (i - 1) * 4 + 1
        mat[i][j] = 1
        j += 4
        mat[i][j] = -1
        vec[i] = -5
    # lb / uf can be float or array
    return opt.LinearConstraint(A=mat, lb=-np.inf, ub=vec, keep_feasible=False)


def opt_display(xk, convergence):
    # comment if you want to print verbose traces
    l = len(xk) // 4
    print(f"IIR    Hz.  Q.   dB [{convergence}]")
    for i in range(l):
        t = int(xk[i * 4 + 0])
        f = int(log_freq[int(xk[i * 4 + 1])])
        q = xk[i * 4 + 2]
        db = xk[i * 4 + 3]
        print(f"{t:3d} {f:5d} {q:1.1f} {db:+1.2f}")

In [ ]:
res = opt.differential_evolution(
    func=opt_peq_flat,
    bounds=opt_bounds_pk(max_peq),
    maxiter=max_iter,
    polish=False,
    integrality=opt_integrality(max_peq),
    callback=opt_display,
    constraints=opt_constraints(max_peq),
    disp=False,
    tol=0.01,
)

In [ ]:
auto_peq = x2peq(res.x)
freq_20 = bisect.bisect(freq, 80)
freq_20k = bisect.bisect(freq, 12000)
auto_peq_spl = peq_spl(freq[freq_20:freq_20k], auto_peq)

fig = pd.DataFrame(
    {
        "Freq": freq[freq_20:freq_20k],
        "target": -lw[freq_20:freq_20k],
        "eq": auto_peq_spl,
        "error": lw[freq_20:freq_20k] + auto_peq_spl,
    }
).plot.line(x="Freq", y=["target", "eq", "error"])
fig.update_xaxes(type="log", title={"text": "Freq (Hz)"})
fig.update_yaxes(title={"text": "SPL"})
fig.update_layout(
    title="{}".format(speaker_name),
    legend={
        "orientation": "v",
        "title": None,
    },
    height=500,
)
fig.show()

In [ ]:
apo = peq_format_apo("experiments", auto_peq)
for iir in apo.split("\n"):
    print(iir)

In [ ]:
opt_peq_flat(res.x)

In [ ]:
-opt_peq_score(res.x)